## Get User Lists

In [ ]:
import datetime
import os

import pandas as pd
from tqdm import tqdm

In [ ]:
def get_mal_header(media):
    if media == "anime":
        return [
            "uid",
            "status",
            "score",
            "num_episodes_watched",
            "is_rewatching",
            "start_date",
            "finish_date",
            "priority",
            "num_times_rewatched",
            "rewatch_value",
            "updated_at",
            "username",
        ]
    elif media == "manga":
        return [
            "uid",
            "status",
            "score",
            "num_volumes",
            "num_chapters",
            "updated_at",
            "username",
        ]
    else:
        assert False

In [ ]:
def get_completion_field(media):
    if media == "anime":
        return "num_episodes_watched"
    elif media == "manga":
        return "num_chapters"
    else:
        assert False

In [ ]:
# Entries in user_{media}_list.csv can be malformed if the notebook crashes in the
# middle of saving a file. This function removes any malformed lines.
def verify_user_media_list_consistency(input_fn, check_user_status=False):
    logger.info(f"Verifying consistency of entries in {input_fn}")
    partition = input_fn.split(".")[-2]
    output_fn = input_fn + "~"
    if check_user_status:
        user_status = pd.read_csv(
            os.path.join(source_dir, f"user_{media}_facts/user_status.{partition}.csv"),
            keep_default_na=False,
        )
        successful_users = set(user_status.loc[lambda x: x["success"]]["username"])
        invalid_users = set()
    with open(input_fn, "r") as in_file:
        with open(output_fn, "w") as out_file:
            header = False
            for line in tqdm(in_file):
                if not header:
                    header = True
                    correct_header = ",".join(get_mal_header(media)) + "\n"
                    if line.strip() != correct_header.strip():
                        logger.warning(
                            f"Replacing malformed header line {line.strip} "
                            f"with correct header {correct_header.strip()}"
                        )
                        line = correct_header
                    out_file.write(line)
                    continue
                fields = line.strip().split(",")
                if len(fields) != len(get_mal_header(media)):
                    logger.warning(
                        f"Deleting malformed line in user_{media}_list.csv {line} "
                    )
                    continue
                if check_user_status:
                    username = get_mal_header(media).index("username")
                    if username not in successful_users:
                        if username not in invalid_users:
                            invalid_users.add(username)
                            logger.warning(
                                f"Deleting entries in user_{media}_list.csv for "
                                f" unrecognized username {username}"
                            )
                        continue
                out_file.write(line)
    os.replace(output_fn, input_fn)

In [ ]:
def to_unix_time(date, fmt):
    return str(int(datetime.datetime.timestamp(datetime.datetime.strptime(date, fmt))))

In [ ]:
def process_uid(uid):
    assert uid.isdigit()
    return uid


def process_status(status):
    if status == "completed":
        return "5"
    elif status in ["watching", "reading"]:
        return "4"
    elif status == "on_hold":
        return "3"
    elif status == "dropped":
        return "2"
    elif status in ["plan_to_watch", "plan_to_read"]:
        return "1"
    elif status == "":
        return "0"
    else:
        assert False


def isfloat(x):
    try:
        float(x)
        return True
    except Exception as e:
        return False


def process_score(score):
    assert isfloat(score)
    parsed_score = float(score)
    assert parsed_score >= 0 and parsed_score <= 10
    return score


def process_num_episodes_watched(num):
    assert num.isdigit()
    return num


def process_is_rewatching(rewatching):
    if rewatching == "-1":
        return "0"
    else:
        assert False


def process_start_date(date):
    if len(rewatching) == 0:
        return "0"
    else:
        return to_unix_time(date, "%Y-%m-%d")


def process_finish_date(date):
    if len(rewatching) == 0:
        return "0"
    else:
        return to_unix_time(date, "%Y-%m-%d")


def process_priority(priority):
    assert priority == "-1"
    return priority


def process_num_times_rewatched(rewatch):
    return rewatch


def process_rewatch_value(value):
    return value


def process_updated_at(time):
    # this should be %Y-%m-%dT%H:%M:%S+00:00" but sometimes the dash
    # is corrupted and replaced with a different character
    time = time[0:4] + "-" + time[5:7] + "-" + time[8:]
    return to_unix_time(time, "%Y-%m-%dT%H:%M:%S+00:00")


def process_username(username):
    return username

In [ ]:
def process_header(header):
    assert header.split(",") == get_mal_header(media)
    return ",".join(
        [
            "username",
            f"{media}id",
            "score",
            "timestamp",
            "status",
            "episodes",
            "source",
        ]
    )


def process_line(line):
    header = get_mal_header(media)
    completion_field = get_completion_field(media)
    try:
        fields = line.split(",")
        media_uid = process_uid(fields[header.index("uid")])
        status = process_status(fields[header.index("status")])
        score = process_score(fields[header.index("score")])
        episodes = process_num_episodes_watched(fields[header.index(completion_field)])
        timestamp = process_updated_at(fields[header.index("updated_at")])
        username = process_username(fields[header.index("username")])
        source = "MAL"
    except Exception as e:
        print(f"Error: could not parse {line}")
        raise e
    return ",".join(
        [username, media_uid, score, timestamp, status, episodes, source]
    )

In [ ]:
def process_user_media_lists(infile, outfile):
    logger.info(f"processing entries in {infile}")
    needs_header = not os.path.exists(outfile)
    with open(infile, "r") as in_file:
        with open(outfile, "a") as out_file:
            header = False
            for line in tqdm(in_file):
                line = line.strip()
                if not header:
                    header = True
                    if needs_header:
                        out_file.write(process_header(line) + "\n")
                    continue
                out_file.write(process_line(line) + "\n")